Model Selection

In [20]:
from typing import Optional, Literal
from pydantic import BaseModel, Field
from openai import OpenAI
import os
import logging


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4.1-nano-2025-04-14"


Class definition of the desired output

In [21]:
class Company_info(BaseModel):
    """First LLM call: Extract basic event information"""
    CIK_code: str = Field(..., description="The 10-digit CIK code number.")
    company_name: str = Field(..., description="The name of the company.")
    description: str = Field(..., description="A brief description of the company.")
    brief_analysis: str = Field(..., description="A brief analysis of the company's recent SEC filings.")

class CIK_CODE(BaseModel):
    CIK_code: str = Field(..., description="The 10-digit CIK code number.")
    warnings: str = Field(..., description="Warnings or notes about the CIK code retrieval process such as ambiguity or multiple matches.")
    

Tool that given a CIK code get's SEC filing data

In [22]:
import requests
import json

def get_SEC(CIK, headers):
    response = requests.get(f"https://data.sec.gov/submissions/CIK{CIK}.json", headers=headers)
    data = response.json()
    
    company_name = data.get("name", "Unknown")
    filings = data.get("filings", {}).get("recent", {})
    
    return {
        "CIK_code": CIK,
        "company_name": company_name,
        "filings": filings
    }



In [ ]:
Name=input("Insert the name of the user: ")

In [ ]:
mail=input("Insert the email of the user: ")

In [ ]:
user_input=input("Insert the name of the company or the CIK code:")

In [26]:
headers = {
    "User-Agent": "MySecScraper/1.0 ({Name}; {mail})"
}


In [41]:
def get_CIK(user_input,headers) -> CIK_CODE:
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-nano-2025-04-14",
        messages=[
        {"role": "system", "content": "given a name of a company, return its CIK code using the tool provided. If ambigous or multiple matches, return a warning message. Also in case of ambiguity, return the CIK code of the most relevant company."},
        {"role": "user", "content": f"{user_input}, {headers}"},
        ],
        response_format=CIK_CODE
        )
    result = completion.choices[0].message.parsed

    return result


In [42]:
def extract_company_info(user_input,headers) -> Company_info:
    
    Input_cik= get_CIK(user_input, headers)
    
    print(Input_cik.warnings)

    SEC_INFO = get_SEC(Input_cik.CIK_code, headers)
    
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"Task: Extract company basics and recent filings summary (≤100 words) from this SEC JSON: CIK, company name, recent filings. Output: Factual sentences for financial reporting.",
            },
            {"role": "user", "content": user_input},
        ],
        response_format=Company_info,
    )
    result = completion.choices[0].message.parsed

    return result


In [46]:
Comp_info= extract_company_info(user_input, headers)

In [47]:
print(Comp_info.CIK_code)
print(Comp_info.company_name)
print(Comp_info.description)
print(Comp_info.brief_analysis)

0000021344
The Coca-Cola Company
The Coca-Cola Company is a multinational corporation known for its production of nonalcoholic beverage concentrates and syrup.
Recent SEC filings indicate the company maintains strong financial health with consistent revenue streams from global operations and ongoing investments in product innovation.
